In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import pickle
import os
import glob
import random
from ssd import default_model

Using TensorFlow backend.


In [2]:
model = default_model()
model.load_weights('weights_SSD300.hdf5')

In [86]:
test_img_list = [mpimg.imread(path) for path in glob.glob('../test_images/*.jpg')]
test_img_list = np.array(test_img_list)

In [4]:
from ssd_utils import BBoxUtility

prior_box = pickle.load(open('prior_boxes_ssd300.pkl', 'rb'))
bbox_utility = BBoxUtility(21, priors=prior_box)

In [108]:
def blending(img, results, confidence=0.6):
    det_label = results[0][:, 0]
    det_conf = results[0][:, 1]
    det_xmin = results[0][:, 2]
    det_ymin = results[0][:, 3]
    det_xmax = results[0][:, 4]
    det_ymax = results[0][:, 5]
    
    top_indices = det_conf >= confidence
    
    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices]
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]
    
    cpimg = np.copy(img)
    
    for i in range(top_conf.shape[0]):
        xmin = int(round(top_xmin[i] * img.shape[1]))
        ymin = int(round(top_ymin[i] * img.shape[0]))
        xmax = int(round(top_xmax[i] * img.shape[1]))
        ymax = int(round(top_ymax[i] * img.shape[0]))

        score = top_conf[i]
        label = int(top_label_indices[i])
        if label != 7:
            continue
            
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.rectangle(cpimg, pt1=(xmin, ymin), pt2=(xmax+5, ymax+5), color=(0, 0, 255), thickness=3)
        cv2.putText(cpimg, '{:0.2f}'.format(score), (xmin+15, ymin-15), font, 1.5, (0, 0, 255), 3, cv2.LINE_AA)
        
    return cpimg

In [110]:
from keras.applications.imagenet_utils import preprocess_input

def process(img):
    test_prediction = model.predict(preprocess_input(np.expand_dims(cv2.resize(img, (300, 300)).astype(np.float32), axis=0)), batch_size=1)
    test_prediction = bbox_utility.detection_out(test_prediction)
    return blending(img, test_prediction, confidence=0.6)

        
# test_img = test_img_list[random.randint(0, test_img_list.shape[0]-1)]
# process(test_img)

In [ ]:
from moviepy.editor import VideoFileClip

clip = VideoFileClip('../project_video.mp4').fl_image(process)
# clip = clip.set_end(10)
# clip.write_gif('./result.gif')
clip.write_videofile('./result.mp4', audio=False)

[MoviePy] >>>> Building video ./result.mp4
[MoviePy] Writing video ./result.mp4


 23%|██▎       | 286/1261 [17:16<59:14,  3.65s/it]  